In [6]:
!cat .env

STAGE=CCR_HOMOLOG


In [7]:
Company.objects.all()

<QuerySet [<Company: 385c3404-2c8b-4987-8015-9b93e044aa6d: CCR - AutoBAn>, <Company: be176150-b4da-4162-9eec-67f6db3f5612: CCR - RodoAnel>, <Company: 161ab539-eea0-4c7c-8e9f-99058261f850: CCR - SPVias>, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>]>

# Criar idCcrAntt Inicio

In [4]:
from tqdm.notebook import tqdm
from helpers.histories import bulk_update_with_history

In [5]:
occs=[]
for a in OccurrenceType.objects.filter(company__name="CCR - Rio Sp"):
    for b in a.form_fields['fields']:
        if b['apiName'] == 'idCcrAntt':
            if b.get('autofill'):
                occs.append(a)

In [2]:
traduzirDirectionValue = lambda value: ''.join([item['name'] for item in Company.objects.get(name='CCR - Rio Sp').custom_options['reporting']['fields']['direction']['selectOptions']['options'] if item['value'].lower() == value.lower()])

In [13]:
reps=[]
for rep in tqdm(Reporting.objects.filter(company__name="CCR - Rio Sp",occurrence_type__in=occs).exclude(occurrence_type__name='Drenagem Superficial')):
     if rep.form_data.get('id_ccr_antt') == '' or rep.form_data.get('id_ccr_antt') == None or not rep.form_data.get('id_ccr_antt'):
        try:
            abrev = next(a.get('autofill').get('cat')[0] for a in rep.occurrence_type.form_fields['fields'] if a['apiName'] == 'idCcrAntt' and a.get('autofill'))
            if not abrev:
                print(rep.occurrence_type.name)
            else:
                if len(str(rep.km).split('.')[1])== 0:
                    metros=str(rep.km).replace(",","+").replace(".","+")+'000'
                elif len(str(rep.km).split('.')[1])== 1:
                    metros=str(rep.km).replace(",","+").replace(".","+")+"00"
                elif len(str(rep.km).split('.')[1])== 2:
                    metros=str(rep.km).replace(",","+").replace(".","+")+"0"
                else: 
                    metros=str(rep.km).replace(",","+").replace(".","+")
                rep.form_data['id_ccr_antt'] = f'''{abrev}{rep.road_name[3:]} {metros} {traduzirDirectionValue(rep.direction)[0]}{" "+str(rep.form_data.get('index')) if rep.form_data.get('index') else ""}'''
                reps.append(rep)
    #             print(f'''{abrev}{rep.road_name[3:]} {metros} {traduzirDirectionValue(rep.direction)[0]}{" "+str(rep.form_data.get('index')) if rep.form_data.get('index') else ""}''')
            abrev=''
        except Exception as e:
            print(rep.number,rep.occurrence_type.name, e)
#     else: 
#         pass

  0%|          | 0/23899 [00:00<?, ?it/s]

In [15]:
user=User.objects.get(username="rlcs")

In [16]:
apontamentos = bulk_update_with_history(reps,Reporting, use_django_bulk=True,batch_size=500,user=user)

# Criar idCcrAntt Fim

# Criar idCcrAntt Inicio para Drenagem Superficial 

In [17]:
from tqdm.notebook import tqdm
from helpers.histories import bulk_update_with_history

In [18]:
traduzirDirectionValue = lambda value: ''.join([item['name'] for item in Company.objects.get(name='CCR - Rio Sp').custom_options['reporting']['fields']['direction']['selectOptions']['options'] if item['value'].lower() == value.lower()])

In [26]:
reps=[]
for rep in tqdm(Reporting.objects.filter(company__name="CCR - Rio Sp",occurrence_type__name='Drenagem Superficial').exclude(form_data__has_key='id_ccr_antt')):
# for rep in tqdm(teste):
    if rep.form_data.get('id_ccr_antt') == '' or rep.form_data.get('id_ccr_antt') == None or not rep.form_data.get('id_ccr_antt'):
        try:
            if rep.form_data.get('elemento') in ['1','2']:
                abrev='DA '
            elif rep.form_data.get('elemento') in ['3']:
                abrev='DI '
            elif rep.form_data.get('elemento') in ['4']:
                abrev='MF '
            elif rep.form_data.get('elemento') in ['5']:
                abrev='SJ '
            elif rep.form_data.get('elemento') in ['6']:
                abrev='VA '
            elif rep.form_data.get('elemento') in ['7']:
                abrev='TSS '
#             abrev = next(a.get('autofill').get('cat')[0] for a in rep.occurrence_type.form_fields['fields'] if a['apiName'] == 'idCcrAntt' and a.get('autofill'))
            if not abrev:
                print(rep.occurrence_type.name)
            else:
                if len(str(rep.km).split('.')[1])== 0:
                    metros=str(rep.km).replace(",","+").replace(".","+")+'000'
                elif len(str(rep.km).split('.')[1])== 1:
                    metros=str(rep.km).replace(",","+").replace(".","+")+"00"
                elif len(str(rep.km).split('.')[1])== 2:
                    metros=str(rep.km).replace(",","+").replace(".","+")+"0"
                else: 
                    metros=str(rep.km).replace(",","+").replace(".","+")
                rep.form_data['id_ccr_antt'] = f'''{abrev}{rep.road_name[3:]} {metros} {traduzirDirectionValue(rep.direction)[0]}{" "+str(rep.form_data.get('index')) if rep.form_data.get('index') else ""}'''
#                 print(rep.form_data)
                reps.append(rep)
#                 print(f'''{abrev}{rep.road_name[3:]} {metros} {traduzirDirectionValue(rep.direction)[0]}{" "+str(rep.form_data.get('index')) if rep.form_data.get('index') else ""}''')
            abrev=''
        except Exception as e:
            print(rep.number,rep.occurrence_type.name, e)
    else: 
        pass

  0%|          | 0/8069 [00:00<?, ?it/s]

In [27]:
len(reps),reps[0].form_data['id_ccr_antt']

(8069, 'MF 101 RJ 410+967 S 1')

In [28]:
user=User.objects.get(username="rlcs")

In [29]:
apontamentos = bulk_update_with_history(reps,Reporting, use_django_bulk=True,batch_size=1000,user=user)

# Testes

In [5]:
reps[0].occurrence_type,
obj_serialized = get_obj_serialized(reps[0], is_reporting=True)
type(obj_serialized)

dict

In [6]:
for a in tqdm(reps[:1]):
    obj_serialized = get_obj_serialized(a, is_reporting=True)

    for c in a.occurrence_type.form_fields['fields']:
        if c['apiName'] == 'idCcrAntt':
            autofillCCR = jsonLogic(c['autofill'],obj_serialized)
    print(autofillCCR)

  0%|          | 0/1 [00:00<?, ?it/s]

TypeError: 'float' object is not subscriptable

In [87]:
reps[0].occurrence_type.form_fields['fields'][0]['autofill'].

{'cat': ['OC ',
  {'cat': [{'substr': [{'var': 'roadName'}, 3, -3]}, ' ']},
  {'substr': [{'var': 'roadName'}, -2, 2]},
  ' ',
  {'if': [{'<': [{'var': 'km'}, 10]},
    {'cat': ['00',
      {'substr': [{'var': 'km'}, 0, 1]},
      '+',
      {'if': [{'==': [{'substr': [{'var': 'km'}, 2, 10]}, 0]},
        '000',
        {'==': [{'substr': [{'var': 'km'}, 3, 10]}, 0]},
        {'cat': [{'substr': [{'var': 'km'}, 2, 1]}, '00']},
        {'==': [{'substr': [{'var': 'km'}, 4, 10]}, 0]},
        {'cat': [{'substr': [{'var': 'km'}, 2, 2]}, '0']},
        {'substr': [{'var': 'km'}, 2]}]}]},
    {'and': [{'>=': [{'var': 'km'}, 10]}, {'<': [{'var': 'km'}, 100]}]},
    {'cat': ['0',
      {'substr': [{'var': 'km'}, 0, 2]},
      '+',
      {'if': [{'==': [{'substr': [{'var': 'km'}, 3, 10]}, 0]},
        '000',
        {'==': [{'substr': [{'var': 'km'}, 4, 10]}, 0]},
        {'cat': [{'substr': [{'var': 'km'}, 3, 1]}, '00']},
        {'==': [{'substr': [{'var': 'km'}, 5, 10]}, 0]},
        {'cat'

In [79]:
for b in range(26):
    for a in tqdm(reps[:1]):
        occ=a.occurrence_type
        for c in occ.form_fields['fields']:
            if c['apiName'] == 'idCcrAntt':
                autofillCCR = jsonLogic(c['autofill'])
        print(autofillCCR)

  0%|          | 0/1 [00:00<?, ?it/s]

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [51]:
reps[9].form_data

{'uf': 'RJ',
 'city': 'Angra dos Reis',
 'zone': '2',
 'ocupation_kind': '5',
 'characteristics': '3',
 'ocupation_kind_others': 'Edificação',
 'id_ccr_antt': 'OC 101 RJ 458+280 N'}

In [6]:
Reporting.objects.first().occurrence_type.form_fields['fields'][0]['autofill'],Reporting.objects.first().occurrence_type.name

({'cat': [{'if': [{'==': [1, {'var': 'formData.elemento'}]},
     'DA ',
     {'==': [2, {'var': 'formData.elemento'}]},
     'DA ',
     {'==': [3, {'var': 'formData.elemento'}]},
     'DI ',
     {'==': [4, {'var': 'formData.elemento'}]},
     'MF ',
     {'==': [5, {'var': 'formData.elemento'}]},
     'SJ ',
     {'==': [6, {'var': 'formData.elemento'}]},
     'VA ',
     {'==': [7, {'var': 'formData.elemento'}]},
     'TSS ']},
   {'cat': [{'substr': [{'var': 'roadName'}, 3, -3]}, ' ']},
   {'var': 'formData.uf'},
   ' ',
   {'if': [{'<': [{'var': 'km'}, 10]},
     {'cat': [{'substr': [{'var': 'km'}, 0, 1]},
       '+',
       {'if': [{'==': [{'substr': [{'var': 'km'}, 2, 10]}, 0]},
         '000',
         {'==': [{'substr': [{'var': 'km'}, 3, 10]}, 0]},
         {'cat': [{'substr': [{'var': 'km'}, 2, 1]}, '00']},
         {'==': [{'substr': [{'var': 'km'}, 4, 10]}, 0]},
         {'cat': [{'substr': [{'var': 'km'}, 2, 2]}, '0']},
         {'substr': [{'var': 'km'}, 2]}]}]},
     {

In [14]:
rep=Reporting.objects.get(number='CCR-Inv-2023.36830')

In [16]:
rep.form_metadata['id_ccr_antt'] = {'manually_specified': False}

In [17]:
rep.save()

Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
